# First Things First

Welcome to our very first introduction to *Tensor Fox*, a specialized library made to deal with tensors of any order (the number of indexes), with focus on the CPD (Canonical Polyadic Decomposition). In order to have everything working properly, all files of Tensor Fox must be in the same folder of your program. Another option is to add all modules to your Python path, if more convenient. To be able to use Tensor Fox you will need the following modules:

- numpy

- scipy

- time

- matplotlib

- numba

- decimal

Also make sure Numpy is using a nice version of BLAS. That is all! Tensor Fox is read to go! Let's start importing Tensor Fox and other necessary modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import TensorFox as tfx

# Creating Tensors and Getting Information 

Let's create a little tensor $T$ just to see how Tensor Fox works at its basics. For third order tensors (3D arrays) I use the convention that $T_{ijk}$ refers to the $i$-th row, $j$-column and $k$-section (frontal slice) of $T$. For instance, consider the tensor defined above (the frontal slices of $T$ are showed)

$$T = \left\{ \left[
\begin{array}{cc}
    0 & 1\\
    2 & 3
\end{array}
\right], \quad
\left[
\begin{array}{cc}
    4 & 5\\
    6 & 7
\end{array}
\right] \right\}.$$

Since Numpy's convention is different from ours with regard to third order tensors. This convention may be irrelevant when using the routines of Tensor Fox, but since I build all the modules thinking this way, it is fair that this point is made explicitly. The function **showtens** prints a third order tensor with this particular convention and print tensors of higher order just as Numpy would print. Below we show both conventions with an example of third order tensor. This particular tensor will be our toy model through all lessons.

In [2]:
# Create and print the tensor, which is 2 x 2 x 2.
m = 2
T = np.zeros((m, m, m))
s = 0

for k in range(m):
    for i in range(m):
        for j in range(m):
            T[i,j,k] = s
            s += 1

print('Our view of T:')
print('--------------')          
tfx.disp.showtens(T)
print()
print('Numpy view of T:')
print('----------------')
print(T)

Our view of T:
--------------
[[0. 1.]
 [2. 3.]]

[[4. 5.]
 [6. 7.]]


Numpy view of T:
----------------
[[[0. 4.]
  [1. 5.]]

 [[2. 6.]
  [3. 7.]]]


There are lots of questions we can make about a given tensor $T$. What are its maximum and minimum entries, what are the dimensions of $T$, the rank and multirank, etc. Even in the case of the simple tensor above we can't know all these answers in advance. The function **infotens** shows lots of information about $T$ for your convenience. This function is useful for small tensors, but remember that finding the rank is a NP-hard task, so don't abuse. 

In [3]:
# Let's see some information about T.
tfx.disp.infotens(T)

T is a tensor of dimensions (2, 2, 2)

|T| = 11.832159566199232

max(T) = 7.0

min(T) = 0.0

E[T] = 3.5

E[|T|] = 3.5

1 <= rank(T) <= 4

Generic rank of the tensor space of T = 2

Computing multilinear rank...
------------------------------------
Estimated multirank(T) = 2 , 2 , 2
|T - (U1, U2, U3)*S|/|T| = 0.0

Computing rank...
Start searching for rank
Stops at r = 4  or less
-----------------------------
Testing r = 3
Estimated rank(T) = 3
|T - T_approx|/|T| = 1.439523236170629e-07



# Computing the CPD

Now let's turn to the most important tool of Tensor Fox, the computation of the CPD. As the previous function hinted, $T$ should have rank 3. We can compute the corresponding CPD with the function **cpd**.

In [4]:
# Compute the CPD of T, assuming T has rank 2.
r = 3
factors, T_approx, output = tfx.cpd(T, r)

# 'factors' is the list of the factor matrices associated with the CPD.
X = factors[0]
Y = factors[1]
Z = factors[2]

# Show the CPD computed. 
print('X =\n', X)
print()
print('Y =\n', Y)
print()
print('Z =\n', Z)
print()

# Show the coordinate representation of this CPD.
print('T_approx =')
tfx.disp.showtens(T_approx)

# Show relative error of this approximation.
print('|T - T_approx|/|T| =', output.rel_error)

X =
 [[-0.76402529  1.45265512  0.39536087]
 [-1.14945451  1.11280149  1.77842897]]

Y =
 [[ 0.52225638  1.34541518  1.47254883]
 [-1.27758702  1.24031932  1.07271607]]

Z =
 [[ 0.83063184 -0.13815892  1.03309272]
 [ 1.10228448  1.824677    1.50061296]]

T_approx =
[[-3.04483394e-06  1.00000328e+00]
 [ 2.00000219e+00  2.99999724e+00]]

[[4.00000179 4.99999761]
 [5.9999981  7.00000166]]

|T - T_approx|/|T| = 5.83244866700309e-07


If it is convenient to use normalized factors, we can convert everything to the format $\Lambda, X, Y, Z$, where the columns of $X, Y, Z$ are unit norm and their magnitudes are passed to the central tensor $\Lambda \in \mathbb{R}^{r \times r \times r}$. Since $\Lambda$ is a diagonal tensor, we can store its values as a unidimensional array of size $r$. This is made such that
$$T \approx \sum_{\ell=1}^r \Lambda_{\ell} \ X_{:, \ell} \otimes Y_{:, \ell} \otimes Z_{:, \ell}.$$

We denote this relation as $T \approx (X, Y, Z) \cdot \Lambda$, the multilinear multiplication. With a simple command we can obtain this factorization.

In [5]:
Lambda, factors = tfx.aux.normalize(factors, r)
X = factors[0]
Y = factors[1]
Z = factors[2]

print('Lambda =\n', Lambda)
print()
print('X =\n', X)
print()
print('Y =\n', Y)
print()
print('Z =\n', Z)
print()

Lambda =
 [2.62927337 6.12748247 6.04692245]

X =
 [[-0.55355718  0.79384399  0.21701124]
 [-0.83281117  0.60812147  0.97616911]]

Y =
 [[ 0.378389    0.73523973  0.80827331]
 [-0.92564667  0.67780716  0.58880748]]

Z =
 [[ 0.60181544 -0.0755008   0.56705846]
 [ 0.7986352   0.99714574  0.82367755]]



At this point you already noted that there are some submodules to TensorFox. In particular, *aux* refers to the *Auxiliar* module, which is responsible for minor subroutines or just some pieces of an important algorithm. 